<a href="https://colab.research.google.com/github/KashanHumayun/how-brainlike-is-deepseek/blob/main/notebooks/01_GPT2_RSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>